Inferential statistics
=====

##### Important concepts
* ENG - Electrical Number Group - a grouping used to divide product into rough categories.
* ETIM class - a grouping used to divide products into detailed categories.
* Technical description - a text field describing each product.

In [2]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

%config InlineBackend.figure_format = 'retina'

# Change working directory to get the file
os.chdir('/Users/pkg/Springboard')

# Open pickled file from the data wrangling section and set working directory
with open('data_storytelling.pickle', 'rb') as handle:
    df = pickle.load(handle)
    
os.chdir('/Users/pkg/Springboard/Intermediate Data Science with Python/Python_Capstone')

In [3]:
df.describe()

,EN_group,ETIM_class,Technical_description
count,201274,201274,201274
unique,41,1813,109426
top,43,EC000042,"Spasial serien består av stål Bokser typeS44,S..."
freq,20721,5630,1772


In [4]:
vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform(df['Technical_description'])

In [5]:
print(vectorizer.get_feature_names())

['00', '000', '0000', '00000', '00001', '00002', '0001', '000120', '0002', '000294', '0003', '000300', '000317', '000324', '0004', '0005', '000550', '000600', '000660', '0007', '0008', '000800', '000860', '0009', '000a', '000cd', '000h', '000k', '000lm', '000lumen', '000lux', '000m', '000rpm', '000t', '000tim', '000timer', '000timerl70', '000timers', '000v', '000va', '000vac', '000w', '001', '0011', '0013', '0015', '0016', '0016µs', '0017', '0018', '0019', '001a', '001v', '002', '0020', '0021', '0021x', '0022', '0023', '0024', '0025', '00259', '0026', '0027', '0028', '0029', '003', '0030', '0031', '0032', '0033', '0034', '0035', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045', '00451', '00452', '0046', '0047', '0048', '0049', '004a', '005', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '005a', '006', '0060', '0061', '0062', '0063', '0065', '0067', '0069', '006p', '0072', '0075', '007638', '008', '0086', '009', '00900', '00902', '00904', 

In [6]:
print(response)

  (0, 90522)	0.287957945681
  (0, 32279)	0.163676418548
  (0, 37965)	0.287957945681
  (0, 58234)	0.113082388457
  (0, 82028)	0.215947849288
  (0, 47542)	0.172888362704
  (0, 29895)	0.172920827047
  (0, 75998)	0.20578212431
  (0, 23327)	0.227576799027
  (0, 21095)	0.0713727495891
  (0, 36714)	0.220934756695
  (0, 63434)	0.0534585262677
  (0, 36393)	0.174469946614
  (0, 37598)	0.144783401114
  (0, 50231)	0.20073279167
  (0, 53403)	0.150250746633
  (0, 36747)	0.106292493941
  (0, 39633)	0.181990542478
  (0, 53897)	0.270605530781
  (0, 60915)	0.159608013007
  (0, 50332)	0.266839287551
  (0, 48683)	0.0830032913788
  (0, 23010)	0.124342992197
  (0, 88797)	0.13398668777
  (0, 78234)	0.136815552765
  :	:
  (201273, 54538)	0.117284165769
  (201273, 50862)	0.0914510728591
  (201273, 4479)	0.108246103683
  (201273, 54790)	0.193714992511
  (201273, 68499)	0.0813578270326
  (201273, 32536)	0.104640110429
  (201273, 22474)	0.134973467955
  (201273, 55139)	0.127369105334
  (201273, 21104)	0.101004887

In [8]:
dictionary = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [16]:
print(max(dictionary, key=dictionary.get), dictionary[max(dictionary, key=dictionary.get)])

00000 12.5192802306


In [17]:
response

<201274x94697 sparse matrix of type '<class 'numpy.float64'>'
	with 6031617 stored elements in Compressed Sparse Row format>

In [18]:
df = pd.DataFrame(response.toarray(), columns= vectorizer.get_feature_names())

In [21]:
df['00000'].value_counts()

0.000000    201273
0.221932         1
Name: 00000, dtype: int64